<a href="https://colab.research.google.com/github/T5d0AfP/text-to-speech/blob/main/Copy_of_Tacotron_Training_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import cv2
from scipy import signal
from scipy import ndimage
import math
import scipy
import matplotlib.pyplot as plt
import skimage.morphology
from skimage.morphology import convex_hull_image, erosion
from skimage.morphology import square
import os

In [6]:
def imageShow( img, title ):
    plt.imshow(img)
    plt.title( title )
    plt.show()

In [7]:
class FingerprintImageEnhancer( object ):
    def __init__( self ):
        self.ridge_segment_blksze = 16
        self.ridge_segment_thresh = 0.1
        self.gradient_sigma = 1
        self.block_sigma = 7
        self.orient_smooth_sigma = 7
        self.ridge_freq_blksze = 38
        self.ridge_freq_windsze = 5
        self.min_wave_length = 5
        self.max_wave_length = 15
        self.kx = 0.65
        self.ky = 0.65
        self.angleInc = 3
        self.ridge_filter_thresh = -3

        self._mask = []
        self._normim = []
        self._orientim = []
        self._mean_freq = []
        self._median_freq = []
        self._freq = []
        self._freqim = []
        self._binim = []
    
    def __normalise( self, img, mean, std ):
        if( np.std( img ) == 0 ):
            raise ValueError( "Image standard devation is 0. Please review Image again" )
        normed = ( img - np.mean( img ) ) / (np.std(img))
        return (normed)
    
    def __ridge_segment(self, img):
        rows, cols = img.shape
        im = self.__normalise(img, 0, 1)

        new_rows = int( self.ridge_freq_blksze * np.ceil( (float(rows)) / (float(self.ridge_segment_blksze)) ))
        new_cols = int( self.ridge_segment_blksze * np.ceil( (float(cols)) / (float(self.ridge_segment_blksze)) ))

        padded_img = np.zeros( (new_rows, new_cols) )
        stddevim = np.zeros( (new_rows, new_cols) )
        padded_img[0:rows][:, 0:cols] = im
        
        for i in range( 0, new_rows, self.ridge_segment_blksze ):
            for j in range( 0, new_cols, self.ridge_segment_blksze ):
                block = padded_img[ i:i + self.ridge_segment_blksze ][:, j:j + self.ridge_segment_blksze]
                stddevim[ i:i + self.ridge_segment_blksze ][:, j:j + self.ridge_segment_blksze] = np.std( block) * np.ones( block.shape )
        
        stddevim = stddevim[ 0:rows ][:, 0:cols ]
        self._mask = stddevim > self.ridge_segment_thresh
        mean_val = np.mean( im[ self._mask ])
        std_val = np.std( im[ self._mask ])
        self._normim = ( im - mean_val ) / ( std_val )

    def __ridge_orient( self ):
        rows, cols = self._normim.shape

        sze = np.fix( 6 * self.gradient_sigma )
        if np.remainder( sze, 2 ) == 0:
            sze = sze + 1

        gauss = cv2.getGaussianKernel( int(sze), self.gradient_sigma )    
        f = gauss * gauss.T

        fy, fx = np.gradient( f )

        Gx = signal.convolve2d( self._normim, fx, mode='same' )
        Gy = signal.convolve2d( self._normim, fy, mode='same')

        Gxx = np.power( Gx, 2 )
        Gyy = np.power( Gy, 2 )
        Gxy = Gx * Gy

        sze = np.fix( 6 * self.block_sigma )

        gauss = cv2.getGaussianKernel( int( sze ), self.block_sigma )
        f = gauss * gauss.T

        Gxx = ndimage.convolve( Gxx, f )
        Gyy = ndimage.convolve( Gyy, f )
        Gxy = 2 * ndimage.convolve( Gxy, f )

        denom = np.sqrt( np.power(Gxy, 2) + np.power( (Gxx - Gyy ),2 )) + np.finfo( float ).eps
        
        sin2theta = Gxy/ denom
        cos2theta = ( Gxx - Gyy ) / denom

        if self.orient_smooth_sigma:
            sze = np.fix( 6 * self.orient_smooth_sigma )
            if np.remainder( sze, 2 ) == 0:
                sze = sze + 1
            gauss = cv2.getGaussianKernel( int( sze ), self.orient_smooth_sigma )
            f = gauss * gauss.T
            cos2theta = ndimage.convolve( cos2theta, f )
            sin2theta = ndimage.convolve( sin2theta, f )

        self._orientim = np.pi / 2 + np.arctan2( sin2theta, cos2theta ) / 2
    
    def __ridge_freq( self ):
        rows, cols = self._normim.shape
        freq = np.zeros((rows, cols))

        for r in range(0, rows - self.ridge_freq_blksze, self.ridge_freq_blksze):
            for c in range(0, cols - self.ridge_freq_blksze, self.ridge_freq_blksze):
                blkim = self._normim[r:r + self.ridge_freq_blksze][:, c:c + self.ridge_freq_blksze]
                blkor = self._orientim[r:r + self.ridge_freq_blksze][:, c:c + self.ridge_freq_blksze]

                freq[r:r + self.ridge_freq_blksze][:, c:c + self.ridge_freq_blksze] = self.__frequest(blkim, blkor)

        self._freq = freq * self._mask
        freq_1d = np.reshape(self._freq, (1, rows * cols))
        ind = np.where(freq_1d > 0)

        ind = np.array(ind)
        ind = ind[1, :]

        non_zero_elems_in_freq = freq_1d[0][ind]

        self._mean_freq = np.mean(non_zero_elems_in_freq)
        self._median_freq = np.median(non_zero_elems_in_freq)  

        self._freq = self._mean_freq * self._mask

    def __frequest( self, blkim, blkor ):
        rows, cols = np.shape(blkim)
        
        cosorient = np.mean( np.cos( 2 * blkor ))
        sinorient = np.mean( np.sin( 2* blkor ))
        orient = math.atan2( sinorient, cosorient ) / 2
                
        rotim = scipy.ndimage.rotate( blkim, orient / np.pi * 180 + 90, axes=(1, 0), reshape=False, order=3, mode='nearest')

        cropsze = int(np.fix(rows / np.sqrt(2)))
        offset = int(np.fix((rows - cropsze) / 2))
        rotim = rotim[offset:offset + cropsze][:, offset:offset + cropsze]
        
        proj = np.sum(rotim, axis=0)
        dilation = scipy.ndimage.grey_dilation(proj, self.ridge_freq_windsze, structure=np.ones(self.ridge_freq_windsze))

        temp = np.abs(dilation - proj)

        peak_thresh = 2

        maxpts = (temp < peak_thresh) & (proj > np.mean(proj))
        maxind = np.where(maxpts)

        rows_maxind, cols_maxind = np.shape(maxind)        

        if (cols_maxind < 2):
            return(np.zeros(blkim.shape))
        else:
            NoOfPeaks = cols_maxind
            waveLength = (maxind[0][cols_maxind - 1] - maxind[0][0]) / (NoOfPeaks - 1)
            if waveLength >= self.min_wave_length and waveLength <= self.max_wave_length:
                return(1 / np.double(waveLength) * np.ones(blkim.shape))
            else:
                return(np.zeros(blkim.shape))
    
    def __ridge_filter( self ):
        im = np.double( self._normim )
        rows, cols = im.shape
        newim = np.zeros( (rows, cols) )

        freq_ld = np.reshape( self._freq, (1, rows * cols ))
        ind = np.where( freq_ld > 0)

        ind = np.array( ind )
        ind = ind[1, :]

        non_zero_elems_in_freq = freq_ld[0][ind]
        non_zero_elems_in_freq = np.double( np.round( (non_zero_elems_in_freq * 100))) / 100

        unfreq = np.unique( non_zero_elems_in_freq )

        sigmax = 1 / unfreq[0] * self.kx
        sigmay = 1 / unfreq[0] * self.ky

        sze = int( np.round( 3 * np.max( [sigmax, sigmay] ) ))

        x, y = np.meshgrid(np.linspace(-sze, sze, (2 * sze + 1)), np.linspace(-sze, sze, (2 * sze + 1)))

        reffilter = np.exp(-(((np.power(x, 2)) / (sigmax * sigmax) + (np.power(y, 2)) / (sigmay * sigmay)))) * np.cos( 2 * np.pi * unfreq[0] * x)

        filt_rows, filt_cols = reffilter.shape

        angleRange = int(180 / self.angleInc)

        gabor_filter = np.array(np.zeros((angleRange, filt_rows, filt_cols)))

        for o in range(0, angleRange):            
            rot_filt = scipy.ndimage.rotate(reffilter, -(o * self.angleInc + 90), reshape=False)
            gabor_filter[o] = rot_filt
        
        maxsze = int(sze)

        temp = self._freq > 0
        validr, validc = np.where(temp)

        temp1 = validr > maxsze
        temp2 = validr < rows - maxsze
        temp3 = validc > maxsze
        temp4 = validc < cols - maxsze

        final_temp = temp1 & temp2 & temp3 & temp4

        finalind = np.where(final_temp)
        
        maxorientindex = np.round(180 / self.angleInc)
        orientindex = np.round(self._orientim / np.pi * 180 / self.angleInc)

        for i in range(0, rows):
            for j in range(0, cols):
                if (orientindex[i][j] < 1):
                    orientindex[i][j] = orientindex[i][j] + maxorientindex
                if (orientindex[i][j] > maxorientindex):
                    orientindex[i][j] = orientindex[i][j] - maxorientindex
        finalind_rows, finalind_cols = np.shape(finalind)
        sze = int(sze)
        for k in range(0, finalind_cols):
            r = validr[finalind[0][k]]
            c = validc[finalind[0][k]]

            img_block = im[r - sze:r + sze + 1][:, c - sze:c + sze + 1]

            newim[r][c] = np.sum(img_block * gabor_filter[int(orientindex[r][c]) - 1])

        self._binim = newim < self.ridge_filter_thresh


    def save_enhanced_image(self, path):        
        cv2.imwrite(path, (255 * self._binim))

    def enhance(self, img, resize=True):        
        if(resize):
            rows, cols = np.shape(img)
            aspect_ratio = np.double(rows) / np.double(cols)

            new_rows = 350                      
            new_cols = new_rows / aspect_ratio

            img = cv2.resize(img, (int(new_cols), int(new_rows)))

        self.__ridge_segment(img)
        imageShow( self._normim, 'after segment')

        self.__ridge_orient()       
        imageShow( self._orientim, 'after orient')

        self.__ridge_freq()         
        imageShow( self._freq, 'after frequest')

        self.__ridge_filter()       
        imageShow( self._binim, 'after filter')

        return(self._binim)

In [8]:
class MinutiaeFeature(object):
    def __init__(self, locX, locY, Orientation, Type):
        self.locX = locX
        self.locY = locY
        self.Orientation = Orientation
        self.Type = Type

class FingerprintFeatureExtractor(object):
    def __init__(self):
        self._mask = []
        self._skel = []
        self.minutiaeTerm = []
        self.minutiaeBif = []

    def __skeletonize(self, img):
        img = np.uint8(img > 128)
        self._skel = skimage.morphology.skeletonize(img)
        self._skel = np.uint8(self._skel) * 255
        self._mask = img * 255

    def __computeAngle(self, block, minutiaeType):
        angle = []
        (blkRows, blkCols) = np.shape(block)
        CenterX, CenterY = (blkRows - 1) / 2, (blkCols - 1) / 2
        if (minutiaeType.lower() == 'termination'):
            sumVal = 0
            for i in range(blkRows):
                for j in range(blkCols):
                    if ((i == 0 or i == blkRows - 1 or j == 0 or j == blkCols - 1) and block[i][j] != 0):
                        angle.append(-math.degrees(math.atan2(i - CenterY, j - CenterX)))
                        sumVal += 1
                        if (sumVal > 1):
                            angle.append(float('nan'))
            return (angle)

        elif (minutiaeType.lower() == 'bifurcation'):
            (blkRows, blkCols) = np.shape(block)
            CenterX, CenterY = (blkRows - 1) / 2, (blkCols - 1) / 2
            angle = []
            sumVal = 0
            for i in range(blkRows):
                for j in range(blkCols):
                    if ((i == 0 or i == blkRows - 1 or j == 0 or j == blkCols - 1) and block[i][j] != 0):
                        angle.append(-math.degrees(math.atan2(i - CenterY, j - CenterX)))
                        sumVal += 1
            if (sumVal != 3):
                angle.append(float('nan'))
            return (angle)

    def __getTerminationBifurcation(self):
        self._skel = self._skel == 255
        (rows, cols) = self._skel.shape
        self.minutiaeTerm = np.zeros(self._skel.shape)
        self.minutiaeBif = np.zeros(self._skel.shape)

        for i in range(1, rows - 1):
            for j in range(1, cols - 1):
                if (self._skel[i][j] == 1):
                    block = self._skel[i - 1:i + 2, j - 1:j + 2]
                    block_val = np.sum(block)
                    if (block_val == 2):
                        self.minutiaeTerm[i, j] = 1
                    elif (block_val == 4):
                        self.minutiaeBif[i, j] = 1

        self._mask = convex_hull_image(self._mask > 0)
        self._mask = erosion(self._mask, square(5))  # Structuing element for mask erosion = square(5)
        self.minutiaeTerm = np.uint8(self._mask) * self.minutiaeTerm

    def __removeSpuriousMinutiae(self, minutiaeList, img, thresh):
        img = img * 0
        SpuriousMin = []
        numPoints = len(minutiaeList)
        D = np.zeros((numPoints, numPoints))
        for i in range(1,numPoints):
            for j in range(0, i):
                (X1,Y1) = minutiaeList[i]['centroid']
                (X2,Y2) = minutiaeList[j]['centroid']

                dist = np.sqrt((X2-X1)**2 + (Y2-Y1)**2)
                D[i][j] = dist
                if(dist < thresh):
                    SpuriousMin.append(i)
                    SpuriousMin.append(j)

        SpuriousMin = np.unique(SpuriousMin)
        for i in range(0,numPoints):
            if(not i in SpuriousMin):
                (X,Y) = np.int16(minutiaeList[i]['centroid'])
                img[X,Y] = 1

        img = np.uint8(img)
        return(img)

    def __cleanMinutiae(self, img):
        self.minutiaeTerm = skimage.measure.label(self.minutiaeTerm, connectivity=2)
        RP = skimage.measure.regionprops(self.minutiaeTerm)
        self.minutiaeTerm = self.__removeSpuriousMinutiae(RP, np.uint8(img), 10)

    def __performFeatureExtraction(self):
        FeaturesTerm = []
        self.minutiaeTerm = skimage.measure.label(self.minutiaeTerm, connectivity=2)
        RP = skimage.measure.regionprops(np.uint8(self.minutiaeTerm))

        WindowSize = 2  # --> For Termination, the block size must can be 3x3, or 5x5. Hence the window selected is 1 or 2
        FeaturesTerm = []
        for num, i in enumerate(RP):
            (row, col) = np.int16(np.round(i['Centroid']))
            block = self._skel[row - WindowSize:row + WindowSize + 1, col - WindowSize:col + WindowSize + 1]
            angle = self.__computeAngle(block, 'Termination')
            if(len(angle) == 1):
                FeaturesTerm.append(MinutiaeFeature(row, col, angle, 'Termination'))

        FeaturesBif = []
        self.minutiaeBif = skimage.measure.label(self.minutiaeBif, connectivity=2)
        RP = skimage.measure.regionprops(np.uint8(self.minutiaeBif))
        WindowSize = 1  # --> For Bifurcation, the block size must be 3x3. Hence the window selected is 1
        for i in RP:
            (row, col) = np.int16(np.round(i['Centroid']))
            block = self._skel[row - WindowSize:row + WindowSize + 1, col - WindowSize:col + WindowSize + 1]
            angle = self.__computeAngle(block, 'Bifurcation')
            if(len(angle) == 3):
                FeaturesBif.append(MinutiaeFeature(row, col, angle, 'Bifurcation'))
        return (FeaturesTerm, FeaturesBif)

    def extractMinutiaeFeatures(self, img):
        self.__skeletonize(img)
        imageShow( self._skel, 'after skeletonize')

        self.__getTerminationBifurcation()
        imageShow( self.minutiaeTerm, 'get termination')

        self.__cleanMinutiae(img)
        imageShow( self.minutiaeTerm, 'after cleanminutiae')

        FeaturesTerm, FeaturesBif = self.__performFeatureExtraction()
        return(FeaturesTerm, FeaturesBif)

    def showResults(self, FeaturesTerm, FeaturesBif):
        
        (rows, cols) = self._skel.shape
        DispImg = np.zeros((rows, cols, 3), np.uint8)
        DispImg[:, :, 0] = 255*self._skel
        DispImg[:, :, 1] = 255*self._skel
        DispImg[:, :, 2] = 255*self._skel

        for idx, curr_minutiae in enumerate(FeaturesTerm):
            row, col = curr_minutiae.locX, curr_minutiae.locY
            (rr, cc) = skimage.draw.circle_perimeter(row, col, 3)
            skimage.draw.set_color(DispImg, (rr, cc), (0, 0, 255))

        for idx, curr_minutiae in enumerate(FeaturesBif):
            row, col = curr_minutiae.locX, curr_minutiae.locY
            (rr, cc) = skimage.draw.circle_perimeter(row, col, 3)
            skimage.draw.set_color(DispImg, (rr, cc), (255, 0, 0))
        
        imageShow(DispImg, 'output')
        # cv2.imshow('output', DispImg)
        # cv2.waitKey(0)

        
def extract_minutiae_features(img, showResult=False):
    feature_extractor = FingerprintFeatureExtractor()
    FeaturesTerm, FeaturesBif = feature_extractor.extractMinutiaeFeatures(img)

    if(showResult):
        feature_extractor.showResults(FeaturesTerm, FeaturesBif)

    return(FeaturesTerm, FeaturesBif)        


In [ ]:
image_enhancer = FingerprintImageEnhancer()

imageFiles = os.listdir('images/')
for i in range( len(imageFiles) ):
    print(imageFiles[i])
    img = cv2.imread( 'images/' + imageFiles[i] )
    imageShow( img, 'original image' )
    img = cv2.cvtColor( img, cv2.COLOR_BGR2GRAY )
    out = image_enhancer.enhance( img )
    imageShow( out, 'enhanced')
    image_enhancer.save_enhanced_image( 'enhanced/' + imageFiles[i] )

    # img = cv2.imread('enhanced/'+ imageFiles[i], 0)
    FeaturesTerminations, FeaturesBifurcations = extract_minutiae_features(255*out, showResult=True)

